In [2]:
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
# from keras.preprocessing.text import Tokenizer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import pandas as pd
import pickle

In [33]:
# wv_from_text = KeyedVectors.load_word2vec_format('../main/models/nkjp+wiki-forms-all-300-skipg-ns-50.txt', binary=False)

MODEL_PATH='../main/models/nkjp+wiki-forms-all-300-skipg-ns-50.txt'
FACTS_PATH='/Users/mjankowski/workspace/hackcrisis/hackcrisis_app/elastic/coronavirus_ground_truth_v8.csv'


In [35]:
def sentence_to_tokens(sentence, stop_words, all_words):
    try:
        all_tokens = word_tokenize(sentence)
        tokens = [x for x in all_tokens if x not in stop_words]
        tokens = [x for x in tokens if x in all_words]
        
        all_tokens_lower = [x.lower() for x in all_tokens]
        tokens_lower = [x for x in all_tokens_lower if x not in stop_words]
        tokens_lower = [x for x in tokens_lower if x in all_words]
        
        return set(set(tokens) | set(tokens_lower))
    except:
        return []

In [38]:
def generate_facts(model_path, facts_path, out_path):
    wv_from_text = pickle.load(open(model_path, "rb"))
    vocab = wv_from_text.wv.vocab
    all_words = []
    for k, v in vocab.items():
        all_words.append(k)

    stop_words = stopwords.words('polish')
    facts = pd.read_csv(FACTS_PATH)
    facts['tokens'] = facts['text'].apply(sentence_to_tokens, args=(stop_words, all_words))
    facts.to_pickle(out_path)


In [39]:
generate_facts('../main/models/model.pickle', FACTS_PATH, '../main/models/facts.pickle')

/anaconda3/envs/hackcrisis_flask/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [28]:
def calculate_score(query_tokens, candidate_tokens, model):
    if (len(query_tokens) > 0) & (len(candidate_tokens) > 0):
        return model.n_similarity(query_tokens, candidate_tokens)
    else: 
        return 0
    

In [29]:
def find_top_matches(query_sentence, stop_words, all_words, facts, model, k=10):
    query_tokens = sentence_to_tokens(query_sentence, stop_words, all_words)
    scores = [calculate_score(query_tokens, tokens, model) for tokens in facts['tokens']]
    facts_scores = facts[['text', 'source', 'is_true']].copy()
    facts_scores['score'] = scores
    result = facts_scores.sort_values(by=['score'], ascending=False).head(k)
    return result

In [30]:
stop_words, all_words, facts, model = init('../main/models/model.pickle', FACTS_PATH)

/anaconda3/envs/hackcrisis_flask/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [31]:
facts.to_pickle('../main/models/facts.pickle')


In [ ]:
pickle.dump(model, open('../main/models/model.pickle', "wb"))
my_model = pickle.load(open('../main/models/model.pickle', "rb" ))

In [23]:
find_top_matches("zwierzęta komary", stop_words, all_words, facts, model)[['text']].values

array([["Nie ma dotychczas żadnych danych wskazujących na to, że zwierzęta domowe, takie jak psy i koty mogą przenosić zakażenie koronawirusem 2019-nCoV.'"],
       ['Obecnie nie ma dowodów na to, że zwierzęta domowe, takie jak psy lub koty, mogą zostać zarażone nowym koronawirusem.'],
       ['Obecnie nie ma dowodów na to, że zwierzęta domowe, takie jak psy lub koty, mogą zostać zarażone nowym koronawirusem.'],
       ['Koronawirusy występują u zwierząt ale zwierzęta nie chorują na choroby z tym zwiazane'],
       ['Zakażenie korona wirusem może być przenoszone przez komary'],
       ['Antybiotyki nie działąją na wirusy tylko na bakterie.'],
       ['Aktualnie nie ma dowodów na to że koronawirus jest przenoszony przez komary.'],
       ['A przede wszystkim mycie rąk pozwala usunąć z powierzchni rąk m.in. bakterie, wirusy czy grzyby odpowiadające za cały szereg groźnych chorób, jak choćby salmonellę czy grypę.'],
       ['A przede wszystkim mycie rąk pozwala usunąć z powierzchni rąk m.

In [24]:
find_top_matches("Czy koronawirus jest podobny do grypy", stop_words, all_words, facts, model)[['text']].values

array([['Grypa to choroba zakaźna układu oddechowego wywołana zakażeniem wirusem grypy. Grypę mogą powodować 3 rodzaje wirusa (A, B i C) należącego do rodziny ortomyksowirusów. Wirus C powoduje zachorowanie o lżejszym przebiegu, natomiast wirusy A i B są przyczyną ciężkich zachorowań, które mogą osiągać rozmiary epidemii grypy.'],
       ['Grypa to choroba zakaźna układu oddechowego wywołana zakażeniem wirusem grypy. Grypę mogą powodować 3 rodzaje wirusa (A, B i C) należącego do rodziny ortomyksowirusów. Wirus C powoduje zachorowanie o lżejszym przebiegu, natomiast wirusy A i B są przyczyną ciężkich zachorowań, które mogą osiągać rozmiary epidemii grypy.'],
       ['Grypa rozwija się po zakażeniu wirusem grypy, a COVID-19 to choroba, która może rozwinąć się po zakażeniu koronawirusem 2019-nCoV '],
       ['Grypa rozwija się po zakażeniu wirusem grypy, a COVID-19 to choroba, która może rozwinąć się po zakażeniu koronawirusem 2019-nCoV '],
       ["Leczenia zakażenia SARS-CoV-2. odbywa s

In [36]:
find_top_matches("Czy koronawirus zniknie w lato", stop_words, all_words, facts, my_model)[['text']].values

array([['Zimna pogoda i snieg niszczą koronawirusa'],
       ['Wraz z nadejsciem ciepleszych dni zagrozenie koronawirusem zmaleje'],
       ['Korona wirus nie może długo utrzymywać sie poza ogranizmem i w zasadzie od razu ginie'],
       ["Dotykanie przedmiotów jest bezpieczne - wirus nie może także zostać przeniesiony, kiedy człowiek dotknie powierzchni lub przedmiotu, na którym znajduje się wirus (czyli np. ślina osoby chorej)'"],
       ['Gorąca kąpiel po powrocie do domu zmiejsza ryzyko zarazenia wirusem'],
       ['Aby ochronić sie przed zakażeniem: Nie podawaj ręki, nie całuj na powitanie, nie obejmuj,Wirus przenosi się drogą kropelkową – zachowaj dystans od rozmówcy 1–1,5 metra,Jeśli możesz zostań w domu i unikaj dużych skupisk ludzkich, unikaj bliskiego kontaktu z chorymi.'],
       ['Aby ochronić sie przed zakażeniem: Nie podawaj ręki, nie całuj na powitanie, nie obejmuj,Wirus przenosi się drogą kropelkową – zachowaj dystans od rozmówcy 1–1,5 metra,Jeśli możesz zostań w domu i

In [2]:
class Word2VecClient:

    def __init__(self, model_path, facts_path):
        self.wv_from_text = pickle.load(open(model_path, "rb"))
        vocab = self.wv_from_text.wv.vocab
        self.all_words = []
        for k, v in vocab.items():
            self.all_words.append(k)

        self.stop_words = stopwords.words('polish')
        self.facts = pd.read_pickle(facts_path)

    def sentence_to_tokens(self, sentence):
        try:
            all_tokens = word_tokenize(sentence)
            tokens = [x for x in all_tokens if x not in self.stop_words]
            tokens = [x for x in tokens if x in self.all_words]

            all_tokens_lower = [x.lower() for x in all_tokens]
            tokens_lower = [x for x in all_tokens_lower if x not in self.stop_words]
            tokens_lower = [x for x in tokens_lower if x in self.all_words]

            return set(set(tokens) | set(tokens_lower))
        except:
            return []

    def calculate_score(self, query_tokens, candidate_tokens):
        if (len(query_tokens) > 0) & (len(candidate_tokens) > 0):
            return self.wv_from_text.n_similarity(query_tokens, candidate_tokens)
        else:
            return 0

    def find_top_matches(self, query_sentence, k=10):
        query_tokens = self.sentence_to_tokens(query_sentence)
        scores = [self.calculate_score(query_tokens, tokens) for tokens in self.facts['tokens']]
        facts_scores = self.facts[['text', 'source', 'is_true']].copy()
        facts_scores['score'] = scores
        result = facts_scores.sort_values(by=['score'], ascending=False).head(k)
        return result


In [6]:
w2v_client = Word2VecClient('../main/models/model.pickle', '../main/models/facts.pickle')


/anaconda3/envs/hackcrisis_flask/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """


In [7]:
result = w2v_client.find_top_matches("mydło", 10)

In [8]:
result.head()

,text,source,is_true,score
133,Po myciu najlepiej wytrzyj ręce w papierowy rę...,pacjent.gov.pl,True,0.557701
248,Po myciu najlepiej wytrzyj ręce w papierowy rę...,NaN,False,0.557701
250,"Jeśli mydło i woda nie są łatwo dostępne, użyj...",NaN,False,0.529651
135,"Jeśli mydło i woda nie są łatwo dostępne, użyj...",pacjent.gov.pl,True,0.529651
93,Gorące powietrze z suszarki do włosów jest sku...,www.newsweek.com,False,0.508637


In [11]:
my_facts = pd.read_pickle('../main/models/facts.pickle')


In [15]:
my_facts = my_facts.dropna(subset=['text'])
my_facts[my_facts['text'].str.contains("myciu najlepiej")]


,id,text,source,is_true,tokens
133,135,Po myciu najlepiej wytrzyj ręce w papierowy rę...,pacjent.gov.pl,True,"{używasz, często, ręcznik, najlepiej, tkaniny,..."
248,251,Po myciu najlepiej wytrzyj ręce w papierowy rę...,NaN,False,"{używasz, często, ręcznik, najlepiej, tkaniny,..."


In [33]:
stop_words = stopwords.words('polish')
facts_txt = pd.read_csv('/Users/mjankowski/workspace/hackcrisis/hackcrisis_app/elastic/coronavirus_ground_truth_v7.csv')
facts_txt['tokens'] = facts_txt['text'].apply(sentence_to_tokens, args=(stop_words, all_words))



NameError: name 'all_words' is not defined

In [17]:
facts_txt = facts_txt.dropna(subset=['text'])
facts_txt[facts_txt['text'].str.contains("myciu najlepiej")]

,id,text,source,is_true
129,134,Po myciu najlepiej wytrzyj ręce w papierowy rę...,pacjent.gov.pl,True


In [22]:
facts_txt.to_pickle('../main/models/facts2.pickle')
facts2 = pd.read_pickle('../main/models/facts2.pickle')

facts2 = facts2.dropna(subset=['text'])
facts2[facts2['text'].str.contains("myciu najlepiej")]

,id,text,source,is_true
129,134,Po myciu najlepiej wytrzyj ręce w papierowy rę...,pacjent.gov.pl,True
